## GrIS catchments map
Updated map of catchments overlaid on a representative annual mass balance field, to support stochastic SMB model description manuscript.

10 Mar 2023 | EHU

In [ ]:
import shapefile
from netCDF4 import Dataset
import numpy as np
import pyproj as pyproj
from scipy import interpolate
import cartopy.crs as ccrs
from cartopy.io.img_tiles import Stamen
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from datetime import date

In [ ]:
## Read in BedMachine surface to evaluate
gl_bed_path ='/Users/lizz/Documents/GitHub/Data_unsynced/BedMachine-Greenland/BedMachineGreenland-2017-09-20.nc'
fh = Dataset(gl_bed_path, mode='r')
xx = fh.variables['x'][:].copy() #x-coord (polar stereo (70, 45))
yy = fh.variables['y'][:].copy() #y-coord
s_raw = fh.variables['surface'][:].copy() #surface elevation
thick_mask = fh.variables['mask'][:].copy()
ss = np.ma.masked_where(thick_mask !=2, s_raw)#mask values: 0=ocean, 1=ice-free land, 2=grounded ice, 3=floating ice, 4=non-Greenland land
fh.close()

X = xx[::4]
Y = yy[::4]
S = ss[::4, ::4]

In [ ]:
## Read in Mouginot catchments from shapefile
print('Reading in Mouginot catchments')
catchment_fn = '/Users/lizz/Documents/GitHub/Data_unsynced/Greenland-catchments-Mouginot/Greenland_Basins_PS_v1.4.2.'
sf = shapefile.Reader(catchment_fn) 

In [ ]:
highlight_catchment_name = 'KANGERLUSSUAQ'

## plot all together, including disjoint multi-catchments
fig, ax = plt.subplots(1)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-45.0))
ax.set(xlim=(min(X)-100, max(X)+100), ylim=(min(Y)-100, max(Y)+100))
# ax.stock_img()
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAND, facecolor='antiquewhite')
ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
for shape in sf.shapeRecords():
    if len(shape.shape.parts)>1:
        catchment_color='grey'
    else:
        catchment_color='k'
    for i in range(len(shape.shape.parts)): ## plot disjointed parts separately
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        x = [i[0] for i in shape.shape.points[i_start:i_end]]
        y = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(x,y, color=catchment_color)
        ax.fill(x,y, color='white')
    if highlight_catchment_name in shape.record['NAME']:
        ax.fill(x,y, color='b')
# ax.contour(X, Y, S)
plt.show()

Okay, the basic map looks okay.  Now instead of white fill, we would like to plot more content: a representative ice-sheet-wide surface mass balance, to show the accumulation and ablation areas.  Let's load in an SMBMIP field for this purpose.

In [ ]:
## Example SMB field read in for grid
print('Reading in example SMB field')
nhm_smb_path = '/Volumes/GoogleDrive/My Drive/Greenland-data/SMBMIP/dEBM_krebs-monthly-ERA-Interim-2010.nc'
fh2 = Dataset(nhm_smb_path, mode='r')
xlon_nhm = fh2.variables['LON'][:].copy() #x-coord (latlon)
ylat_nhm = fh2.variables['LAT'][:].copy() #y-coord (latlon)
smb_debm = fh2.variables['SMBcorr'][:].copy() ## SMB field
fh2.close()

In [ ]:
smb_annual = sum(smb_debm[m] for m in range(12))

In [ ]:
###------------------------
### SET UP SMB REPROJECTION
###------------------------

## Down-sample bed topo
x_3km = xx[::20] # sample at ~3 km resolution
y_3km = yy[::20]

## Down-sample SMB
x_lon_h = xlon_nhm[::2, ::2] 
y_lat_h = ylat_nhm[::2, ::2] # resolution about 2 km

print('Creating reprojected meshgrid')
wgs84 = pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by SMB data
psn_gl = pyproj.Proj("+init=epsg:3413") # Polar Stereographic North used by BedMachine and Mankoff
xs, ys = pyproj.transform(wgs84, psn_gl, x_lon_h, y_lat_h)
Xmat, Ymat = np.meshgrid(x_3km, y_3km) # Downsampled BedMachine coords

smb_ds = smb_annual[::2,::2] ## downsample
regridded_smb = interpolate.griddata((xs.ravel(), ys.ravel()), smb_ds.ravel(), (Xmat, Ymat), method='nearest')

In [ ]:
fig, ax = plt.subplots()
ax.imshow(regridded_smb)
plt.show()

In [ ]:
highlight_catchment_name = 'KANGERLUSSUAQ'

## plot all together, including disjoint multi-catchments
fig, ax = plt.subplots(1)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-45.0))
ax.set(xlim=(min(X)-100, max(X)+100), ylim=(min(Y)-100, max(Y)+100))
# ax.stock_img()
ax.add_feature(cfeature.OCEAN)
# ax.add_feature(cfeature.LAND, facecolor='antiquewhite')
# ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
ax.contourf(x_3km, y_3km, regridded_smb, levels=10)
# ax.contour(X, Y[::-1], SMB_toshow(X, Y), levels=10)
for shape in sf.shapeRecords():
    if len(shape.shape.parts)>1:
        catchment_color='grey'
    else:
        catchment_color='k'
    for i in range(len(shape.shape.parts)): ## plot disjointed parts separately
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        x = [i[0] for i in shape.shape.points[i_start:i_end]]
        y = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(x,y, color=catchment_color)
#         ax.fill(x,y, color='white')
    if highlight_catchment_name in shape.record['NAME']:
        ax.fill(x,y, color='b')
# ax.contour(X, Y, S)
plt.show()

In [ ]:
np.shape(thick_mask[::20,::20])

We need to mask for only grounded ice in the contour plot above.

In [ ]:
rg_smb = np.ma.masked_where(thick_mask[::20,::20] !=2, regridded_smb)

In [ ]:
rg_smb.max() ## figuring out how to modify the colorbar to show the data range best

In [ ]:
highlight_catchment_name = 'KANGERLUSSUAQ'

divnorm = colors.TwoSlopeNorm(vmin=-6000, vcenter=0, vmax=2000)
# bounds = np.array([-3000, -1500, 0, 1500, 3000])
# bnorm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)


## plot all together, including disjoint multi-catchments
fig, ax = plt.subplots(1, figsize=(4,6))
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-45.0))
ax.set(xlim=(min(X)-100, max(X)+100), ylim=(min(Y)-100, max(Y)+100))
# ax.stock_img()
ax.add_feature(cfeature.OCEAN)
# ax.add_feature(cfeature.LAND, facecolor='antiquewhite') ## these do not match the land mask of bedmachine and smbmip
# ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
c = ax.contourf(x_3km, y_3km, rg_smb, cmap=cm.bwr_r, norm=divnorm, vmin=-6000, vmax=2000, levels=100, extend='both')
for shape in sf.shapeRecords():
    if len(shape.shape.parts)>1:
        catchment_color='grey'
    else:
        catchment_color='k'
    for i in range(len(shape.shape.parts)): ## plot disjointed parts separately
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        x = [i[0] for i in shape.shape.points[i_start:i_end]]
        y = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(x,y, color=catchment_color)
#     if highlight_catchment_name in shape.record['NAME']:
# #         ax.fill(x,y, color='b', alpha=0.5)
#         pass
cbar = fig.colorbar(c, shrink=0.6, extend='both', orientation='horizontal', label='Annual SMB [mm w.e.]')
cbar.ax.set(xticks=(-6000, -3000, 0, 2000))
plt.show()

Okay, now two clean-up items.  
- Plot the main part of disjoint catchments (generally ice-sheet-connected) in black rather than grey, and 
- Remove the bounding box that is showing around the plot.

In [ ]:
highlight_catchment_name = 'KANGERLUSSUAQ'

divnorm = colors.TwoSlopeNorm(vmin=-6000, vcenter=0, vmax=2000)

## plot all together, including disjoint multi-catchments
fig=plt.figure(figsize=(4,6)) ## was doubly defining 'ax' above; subplots command creates bounding box that figure does not

ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=-45.0))
ax.set(xlim=(min(X)-100, max(X)+100), ylim=(min(Y)-100, max(Y)+100))
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAND, facecolor='antiquewhite') ## these do not match the land mask of bedmachine and smbmip
ax.add_feature(cfeature.COASTLINE,linewidth=0.3)

c = ax.contourf(x_3km, y_3km, rg_smb, cmap=cm.bwr_r, norm=divnorm, 
                vmin=-6000, vmax=2000, levels=100, extend='both',
               transform=ccrs.epsg(3413))
for shape in sf.shapeRecords():
    for i in range(len(shape.shape.parts)): ## plot disjointed parts separately
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        if i>1:
            catchment_color='grey'
        else:
            catchment_color='k'
        x = [i[0] for i in shape.shape.points[i_start:i_end]]
        y = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(x,y, color=catchment_color, transform=ccrs.epsg(3413)) ## add transform argument to make these match

# ax_d = make_axes_locatable(ax)
# cax = ax_d.append_axes('bottom', size='7%', pad='2%')
cbar = fig.colorbar(c, extend='both', orientation='horizontal', label='Annual SMB [mm w.e.]', pad=0.02, shrink=0.7)
cbar.ax.set(xticks=(-6000, -3000, 0, 2000))

plt.savefig('/Users/lizz/Documents/Research/StISP/Figure-making/{}-Greenland_catchments_SMB_map-land_underlay'.format(date.today().strftime('%Y%m%d')), 
            dpi=300, format='png')